#### Install important packages

In [1]:
# Core RL libraries and wrappers
!pip install "minari[all]" d3rlpy gymnasium gymnasium_robotics pyvirtualdisplay

# For rendering videos in headless environment (Colab)
!apt-get install -y xvfb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.5/852.5 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 M

#### Import Libraries

In [2]:
import os
import numpy as np
import shutil
import gymnasium as gym
from pyvirtualdisplay import Display
from gymnasium.wrappers import RecordVideo
from d3rlpy.algos import BCQ
import minari
import torch
import d3rlpy

#### Code to render and save video

In [3]:
# Create folder for videos
VIDEO_DIR = "videos"
shutil.rmtree(VIDEO_DIR, ignore_errors=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

In [6]:
# Start virtual display for video rendering
Display(visible=0, size=(640, 480)).start()

def render_models_to_video(models_dir, env_name, video_dir="videos"):
    """
    Renders one episode for each model in the folder and saves it as video.

    Args:
        models_dir (str): Folder with .d3 model files(. => current directory)
        env_name (str): Gymnasium environment name (e.g., 'Hopper-v4')
        algo_class (class): Algorithm class (e.g., BCQConfig)
        video_dir (str): Folder to save videos
    """
    os.makedirs(video_dir, exist_ok=True)

    # selected_steps = ["model_10000.d3", "model_30000.d3", "model_50000.d3", "model_100000.d3"]
    selected_steps = ["model_100000.d3"]

    for model_file in selected_steps:
        model_path = os.path.join(models_dir, model_file)
        print(f"Rendering {model_file}")

        model = d3rlpy.load_learnable(model_path)

        env = gym.make(env_name, render_mode="rgb_array")

        # Set up video recorder
        step_name = model_file.split(".")[0]
        video_path = os.path.join(video_dir, step_name)
        env = RecordVideo(env, video_folder=video_path, name_prefix=step_name)


        obs, _ = env.reset()
        for _ in range(1000):  # Max steps for agent
            obs_input = np.expand_dims(obs, axis=0)
            action = model.predict(obs_input)[0]
            # action = model.predict(np.expand_dims(obs, axis=0))[0]
            obs, reward, terminated, truncated, _ = env.step(action)
            if terminated or truncated:
                break

        env.close()

    print("Video rendering complete!")


render_models_to_video(
    models_dir="./",
    env_name="Hopper-v5",  # change env name as used
    video_dir="videos_hopper_awac"
)

Rendering model_100000.d3
Video rendering complete!


In [7]:
from IPython.display import HTML
from base64 import b64encode

def display_video(path):
    with open(path, "rb") as f:
        video = f.read()
    b64 = b64encode(video).decode()
    return HTML(f'<video controls autoplay loop><source src="data:video/mp4;base64,{b64}" type="video/mp4">')

In [8]:
display_video("videos_hopper_awac/model_100000/model_100000-episode-0.mp4")

In [ ]:
display_video("videos_pendulum_bear/model_10000/model_10000-episode-0.mp4")

In [ ]:
display_video("videos_pendulum_bear/model_30000/model_30000-episode-0.mp4")

In [ ]:
display_video("videos_pendulum_bear/model_50000/model_50000-episode-0.mp4")

In [ ]:
display_video("videos_pendulum_bear/model_100000/model_100000-episode-0.mp4")